In [3]:
#Read in the file
import pandas as pd

data_raw = pd.read_csv('cleaned_data_V3.csv')

data_raw.head()

,C1AccountNo,CXRecords,Year,Territory,Jan,Feb,Mar,Apr,May,Jun,...,Aug,Sep,Oct,Nov,Dec,q1_sales,q2_sales,q3_sales,q4_sales,annual_sales
0,A2061733056$=L._,91470FS$KD_> W<,2018,West,0.00,0.00,0.00,0.0,0.00,0.0,...,0.0,639.00,0.00,0.00,1438.00,0.00,0.00,639.00,1438.00,2077.00
1,A2061733057#*N-2,91470GQ%![I! W<,2018,Central,1730.06,10023.32,108.18,0.0,0.00,0.0,...,0.0,0.00,282.60,0.00,861.14,11861.56,0.00,0.00,1143.74,13005.30
2,A2061733058)U90^,91470I8%.+ML W<,2018,West,0.00,0.00,0.00,0.0,306.86,0.0,...,0.0,0.00,211.37,66.01,0.00,0.00,306.86,0.00,277.38,584.24
3,A2061733059#5)>2,"91470KJ(,V-0 W<",2018,Central,0.00,0.00,0.00,0.0,0.00,0.0,...,0.0,153.46,0.00,0.00,0.00,0.00,0.00,153.46,0.00,153.46
4,A2061733060)(W<%,91470M0#>H/T W<,2018,Midwest,0.00,302.72,0.00,0.0,0.00,0.0,...,0.0,0.00,0.00,0.00,0.00,302.72,0.00,0.00,0.00,302.72


In [5]:
unique_years = data_raw['Year'].unique()
unique_comp_terr = data_raw['Territory'].unique()
print("Unique values in 'Year':", unique_years)
print("Unique values in 'Comp Terr':", unique_comp_terr)

Unique values in 'Year': [2018 2019 2020 2021 2022 2023 2024]
Unique values in 'Comp Terr': ['West' 'Central' 'Midwest' 'Northeast' 'South' 'International' 'Unknown'
 'Special']


In [6]:
# Calculating total sales for the years 2018 and 2019 (test the result)
total_sales_2018 = data_raw[data_raw['Year'] == 2018]['annual_sales'].sum()
total_sales_2019 = data_raw[data_raw['Year'] == 2019]['annual_sales'].sum()

# Calculating the Purchase Rate using the formula
purchase_rate = ((total_sales_2019 - total_sales_2018) / total_sales_2018) * 100


purchase_rate

55.01093958569844

In [7]:
#Table 1
purchase_rates = {}
for year in range(2018, 2024):
    total_sales_current_year = data_raw[data_raw['Year'] == year]['annual_sales'].sum()
    total_sales_next_year = data_raw[data_raw['Year'] == year + 1]['annual_sales'].sum()
    
    # Calculating the purchase rate
    purchase_rate = ((total_sales_next_year - total_sales_current_year) / total_sales_current_year) * 100
    purchase_rates[f"{year}-{year + 1}"] = purchase_rate

purchase_rates

{'2018-2019': 55.01093958569844,
 '2019-2020': -6.477928858294703,
 '2020-2021': 20.752439868577106,
 '2021-2022': 34.4394925399398,
 '2022-2023': 5.27701835518234,
 '2023-2024': -47.48530863305519}

In [12]:
#Table 2
purchase_rate_matrix = {}

# Looping through each consecutive year pair from 2018 to 2023
for year in range(2018, 2024):
    purchase_rate_matrix[f"{year}-{year + 1}"] = {}
    
    #Looping through each region to calculate the purchase rate for that region and year pair
    for region in ['West', 'Central', 'Midwest', 'Northeast', 'South', 'International', 'Special']:
        total_sales_current_year = data_raw[(data_raw['Year'] == year) & (data_raw['Territory'] == region)]['annual_sales'].sum()
        total_sales_next_year = data_raw[(data_raw['Year'] == year + 1) & (data_raw['Territory'] == region)]['annual_sales'].sum()
        
        if total_sales_current_year == 0:
            purchase_rate = None 
        else:
            purchase_rate = ((total_sales_next_year - total_sales_current_year) / total_sales_current_year) * 100
        
        purchase_rate_matrix[f"{year}-{year + 1}"][region] = purchase_rate

purchase_rate_df = pd.DataFrame(purchase_rate_matrix).T

purchase_rate_df

,West,Central,Midwest,Northeast,South,International,Special
2018-2019,75.980481,89.434280,34.742409,33.554379,41.033528,-25.782607,NaN
2019-2020,-21.821305,-12.723205,11.112568,-1.113224,-0.128405,15.124976,NaN
2020-2021,28.802864,2.918958,-39.308389,9.471794,15.998281,521.520643,NaN
2021-2022,6.276435,27.853064,132.762770,81.435894,67.830907,-49.625150,-66.289141
2022-2023,37.363468,10.176378,-39.709103,9.884096,1.345757,166.170133,422.701343
2023-2024,-41.750279,-50.622399,-65.581609,-38.238718,-50.827157,-32.372433,-71.793925


In [10]:
# table 3
monthly_purchase_rate_matrix = {}

for year in range(2018, 2024):
    monthly_purchase_rate_matrix[f"{year}-{year + 1}"] = {}
    
    for month_start, month_end in [('Jan', 'Feb'), ('Feb', 'Mar'), ('Mar', 'Apr'), ('Apr', 'May'),
                                   ('May', 'Jun'), ('Jun', 'Jul'), ('Jul', 'Aug'), ('Aug', 'Sep'),
                                   ('Sep', 'Oct'), ('Oct', 'Nov'), ('Nov', 'Dec')]:
       
        monthly_sum_current_year = data_raw[data_raw['Year'] == year][month_start].sum()
        monthly_sum_next_year = data_raw[data_raw['Year'] == year + 1][month_end].sum()
        
      
        if monthly_sum_current_year == 0:
            purchase_rate = None  
        else:
            purchase_rate = ((monthly_sum_next_year - monthly_sum_current_year) / monthly_sum_current_year) * 100
        
        monthly_purchase_rate_matrix[f"{year}-{year + 1}"][f"{month_start}-{month_end}"] = purchase_rate


monthly_purchase_rate_df = pd.DataFrame(monthly_purchase_rate_matrix).T

monthly_purchase_rate_df

,Jan-Feb,Feb-Mar,Mar-Apr,Apr-May,May-Jun,Jun-Jul,Jul-Aug,Aug-Sep,Sep-Oct,Oct-Nov,Nov-Dec
2018-2019,100.205272,67.613379,18.096813,43.487727,15.350678,57.305804,72.798355,58.316975,108.659674,16.009000,16.617365
2019-2020,57.564726,-6.280757,-98.668019,1.436206,-40.475609,-9.810090,-26.826696,3.225224,1.373748,-16.915472,32.525355
2020-2021,-9.359765,-8.665524,-1.981310,4337.121561,-0.388146,22.076223,64.018957,83.046566,41.112888,31.282753,12.760123
2021-2022,142.464940,104.436617,13.736737,32.873570,48.603590,16.881471,15.799306,-18.390971,-3.175754,-0.866108,1.718604
2022-2023,49.437833,39.046566,-5.343526,-1.836588,-31.424821,-48.373873,-27.147261,-12.364517,4.638401,-4.481201,-33.610653
2023-2024,-36.926879,-21.670841,-49.748032,-50.356520,-54.904354,-53.425532,-80.025510,-100.000000,-100.000000,-100.000000,-100.000000


In [13]:
#Table 4
monthly_purchase_rate_by_region_matrix = {}


for year in range(2018, 2024):
    monthly_purchase_rate_by_region_matrix[f"{year}-{year + 1}"] = {}
    
    
    for region in ['West', 'Central', 'Midwest', 'Northeast', 'South', 'International', 'Special']:
        monthly_purchase_rate_by_region_matrix[f"{year}-{year + 1}"][region] = {}
        
        
        for month_start, month_end in [('Jan', 'Feb'), ('Feb', 'Mar'), ('Mar', 'Apr'), ('Apr', 'May'),
                                       ('May', 'Jun'), ('Jun', 'Jul'), ('Jul', 'Aug'), ('Aug', 'Sep'),
                                       ('Sep', 'Oct'), ('Oct', 'Nov'), ('Nov', 'Dec')]:
           
            monthly_sum_current_year = data_raw[(data_raw['Year'] == year) & (data_raw['Territory'] == region)][month_start].sum()
            monthly_sum_next_year = data_raw[(data_raw['Year'] == year + 1) & (data_raw['Territory'] == region)][month_end].sum()
            
           
            if monthly_sum_current_year == 0:
                purchase_rate = None  
            else:
                purchase_rate = ((monthly_sum_next_year - monthly_sum_current_year) / monthly_sum_current_year) * 100
            
            monthly_purchase_rate_by_region_matrix[f"{year}-{year + 1}"][region][f"{month_start}-{month_end}"] = purchase_rate

monthly_purchase_rate_by_region_df = pd.DataFrame.from_dict({(i, j): monthly_purchase_rate_by_region_matrix[i][j] 
                                                             for i in monthly_purchase_rate_by_region_matrix 
                                                             for j in monthly_purchase_rate_by_region_matrix[i]}, orient='index')

monthly_purchase_rate_by_region_df

Jan-Feb      Feb-Mar     Mar-Apr        Apr-May  \
2018-2019 West           188.512890   126.550251   43.798129      56.661222   
          Central         87.176991    53.947786   39.132940      77.627682   
          Midwest        104.127233    76.747661  -24.496626       5.604238   
          Northeast       94.125792    74.045234   30.403301      33.411552   
          South           51.560047    23.897136   18.496909      29.356900   
          International   81.331555   -60.399273  -52.536156      -4.450529   
          Special               NaN          NaN         NaN            NaN   
2019-2020 West            61.913315    -8.156535  -99.489068     -21.678436   
          Central         46.053003   -21.203357 -100.028612     -20.776426   
          Midwest         28.411160   -28.784199  -98.053607      40.830309   
          Northeast       97.752830    31.853727  -99.916504      20.475154   
          South           73.460237    13.571902  -94.845984       8.839247   
          International   96.903339   -12.531679 -100.000000     192.336524   
          Special               NaN          NaN         NaN            NaN   
2020-2021 West            17.952574    -5.652295  -15.469539   11252.774512   
          Central        -29.076004   -11.318950   13.054215 -195266.221220   
          Midwest        -50.282200   -44.102328  -40.722875    1124.646124   
          Northeast      -23.811430   -32.330653  -19.189931   54818.117914   
          South           -8.533598   -34.468149   -8.773848    1229.549841   
          International  534.324649  1297.538846  356.976414            NaN   
          Special               NaN          NaN         NaN            NaN   
2021-2022 West            67.006263    35.376837  -23.000784      -1.442747   
          Central        144.859789    72.902747   -4.259640      19.526817   
          Midwest        259.749389   245.123469  105.803720     148.186995   
          Northeast      217.130787   209.990616  104.251382      64.410829   
          South          256.239865   184.243176   61.113097      63.291514   
          International  -50.455653   -35.470188  -32.993186     -61.550971   
          Special       -100.000000   -67.986733         NaN            NaN   
2022-2023 West            85.491251    75.218254   28.653138      14.583316   
          Central         54.086139    67.959373   -1.605765      19.082504   
          Midwest          9.519134    -2.375789  -42.230753     -57.238519   
          Northeast       73.732548    39.807094    8.807502       6.499461   
          South           25.767667     6.042940  -21.142208       6.045797   
          International  374.193367   552.107043  143.573526      54.554595   
          Special               NaN          NaN  801.315789     409.513794   
2023-2024 West           -41.520992   -20.977138  -36.378824     -47.146078   
          Central        -44.180920   -27.217759  -60.642220     -43.155694   
          Midwest        -51.446990   -52.123393  -71.733266     -73.574782   
          Northeast      -19.372818   -12.393766  -38.010308     -42.751196   
          South          -37.029102   -14.424829  -45.456310     -54.340912   
          International   -1.913670    63.420072  -50.172898     -30.907730   
          Special               NaN          NaN  -94.223602     -94.666360   

                            May-Jun     Jun-Jul      Jul-Aug     Aug-Sep  \
2018-2019 West            56.499096   42.887953    58.027533   69.848631   
          Central         21.307813  115.646105    67.149135  283.296007   
          Midwest         59.712233  -25.391174    33.041183    4.872440   
          Northeast      -11.164338   79.063038    38.508252  -14.698229   
          South           -9.640276   88.899217   171.984370    5.621261   
          International  -67.312013   38.820602    18.096980   91.829133   
          Special               NaN         NaN          NaN         NaN   
2019-2020 West        

In [14]:
import pandas as pd
import psycopg2

In [15]:
# Converting Table 1 (purchase_rate_1) to a DataFrame
purchase_rate_df_1 = pd.DataFrame.from_dict(purchase_rates, orient='index', columns=['purchase_rate']).reset_index()
purchase_rate_df_1.rename(columns={'index': 'year_range'}, inplace=True)

def get_db_connection():
    conn = psycopg2.connect(
        dbname='5900_test', user='postgres', password='123', host='localhost'
    )
    return conn


create_table_query = """
CREATE TABLE IF NOT EXISTS purchase_rate_1 (
    year_range VARCHAR(50),
    purchase_rate FLOAT
);
"""


insert_query = """
INSERT INTO purchase_rate_1 (year_range, purchase_rate) 
VALUES (%s, %s);
"""

try:
    conn = get_db_connection()
    cursor = conn.cursor()

 
    cursor.execute(create_table_query)
    conn.commit()

    
    for _, row in purchase_rate_df_1.iterrows():
        cursor.execute(insert_query, row.tolist())

    
    conn.commit()
    print("Data inserted successfully into the purchase_rate_1 table.")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cursor.close()
        conn.close()

Data inserted successfully into the purchase_rate_1 table.


In [16]:
#Insert Table 2 Into the database
def get_db_connection():
    conn = psycopg2.connect(
        dbname='5900_test', user='postgres', password='123', host='localhost'
    )
    return conn


create_table_query = """
CREATE TABLE IF NOT EXISTS purchase_rate_2 (
    year_range VARCHAR(50),
    West FLOAT,
    Central FLOAT,
    Midwest FLOAT,
    Northeast FLOAT,
    South FLOAT,
    International FLOAT,
    Special FLOAT
);
"""


insert_query = """
INSERT INTO purchase_rate_2 (year_range, West, Central, Midwest, Northeast, South, International, Special) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

# create the year_range index as column
purchase_rate_df.reset_index(inplace=True)
purchase_rate_df.rename(columns={'index': 'year_range'}, inplace=True)

try:
    conn = get_db_connection()
    cursor = conn.cursor()

   
    cursor.execute(create_table_query)
    conn.commit()

    
    for _, row in purchase_rate_df.iterrows():
        cursor.execute(insert_query, row.tolist())

    
    conn.commit()
    print("Data inserted successfully into the purchase_rate_2 table.")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cursor.close()
        conn.close()

Data inserted successfully into the purchase_rate_2 table.


In [17]:
#Insert Table 3
def get_db_connection():
    conn = psycopg2.connect(
        dbname='5900_test', user='postgres', password='123', host='localhost'
    )
    return conn


create_table_query = """
CREATE TABLE IF NOT EXISTS purchase_rate_3 (
    year_range VARCHAR(50),
    "Jan-Feb" FLOAT,
    "Feb-Mar" FLOAT,
    "Mar-Apr" FLOAT,
    "Apr-May" FLOAT,
    "May-Jun" FLOAT,
    "Jun-Jul" FLOAT,
    "Jul-Aug" FLOAT,
    "Aug-Sep" FLOAT,
    "Sep-Oct" FLOAT,
    "Oct-Nov" FLOAT,
    "Nov-Dec" FLOAT
);
"""


insert_query = """
INSERT INTO purchase_rate_3 (year_range, "Jan-Feb", "Feb-Mar", "Mar-Apr", "Apr-May", 
                             "May-Jun", "Jun-Jul", "Jul-Aug", "Aug-Sep", 
                             "Sep-Oct", "Oct-Nov", "Nov-Dec") 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""


monthly_purchase_rate_df.reset_index(inplace=True)
monthly_purchase_rate_df.rename(columns={'index': 'year_range'}, inplace=True)

try:
    conn = get_db_connection()
    cursor = conn.cursor()

   
    cursor.execute(create_table_query)
    conn.commit()

   
    for _, row in monthly_purchase_rate_df.iterrows():
        cursor.execute(insert_query, row.tolist())

    
    conn.commit()
    print("Data inserted successfully into the purchase_rate_3 table.")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cursor.close()
        conn.close()

Data inserted successfully into the purchase_rate_3 table.


In [18]:
#Insert Table 4
def get_db_connection():
    conn = psycopg2.connect(
        dbname='5900_test', user='postgres', password='123', host='localhost'
    )
    return conn


create_table_query = """
CREATE TABLE IF NOT EXISTS purchase_rate_4 (
    year_range VARCHAR(50),
    region VARCHAR(50),
    "Jan-Feb" FLOAT,
    "Feb-Mar" FLOAT,
    "Mar-Apr" FLOAT,
    "Apr-May" FLOAT,
    "May-Jun" FLOAT,
    "Jun-Jul" FLOAT,
    "Jul-Aug" FLOAT,
    "Aug-Sep" FLOAT,
    "Sep-Oct" FLOAT,
    "Oct-Nov" FLOAT,
    "Nov-Dec" FLOAT
);
"""


insert_query = """
INSERT INTO purchase_rate_4 (year_range, region, "Jan-Feb", "Feb-Mar", "Mar-Apr", "Apr-May", 
                             "May-Jun", "Jun-Jul", "Jul-Aug", "Aug-Sep", 
                             "Sep-Oct", "Oct-Nov", "Nov-Dec") 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""


monthly_purchase_rate_by_region_df.reset_index(inplace=True)
monthly_purchase_rate_by_region_df.rename(columns={'level_0': 'year_range', 'level_1': 'region'}, inplace=True)

try:
    conn = get_db_connection()
    cursor = conn.cursor()


    cursor.execute(create_table_query)
    conn.commit()

    
    for _, row in monthly_purchase_rate_by_region_df.iterrows():
        cursor.execute(insert_query, row.tolist())

   
    conn.commit()
    print("Data inserted successfully into the purchase_rate_4 table.")

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        cursor.close()
        conn.close()

Data inserted successfully into the purchase_rate_4 table.
